In [1]:
import pandas as pd
import re
import sys

df = pd.read_csv(f'data/COVIDSenti.csv')
print(df.head())

                                               tweet label
0  Coronavirus | Human Coronavirus Types | CDC ht...   neu
1  @shehryar_taseer That‚Äôs üíØ true , \nCorona...   neu
2  TLDR: Not SARS, possibly new coronavirus. Diff...   neg
3  Disease outbreak news from the WHO: Middle Eas...   neu
4  China - Media: WSJ says sources tell them myst...   neu


In [2]:
df['label'].value_counts()

neu    67385
neg    16335
pos     6280
Name: label, dtype: int64

In [3]:
df['label'] = df['label'].replace({'pos': 1, 'neu': 0, 'neg': -1})

In [4]:
X, y = df['tweet'], df['label']

In [5]:
X = X.apply(lambda x: x.lower())
X.head()

0    coronavirus | human coronavirus types | cdc ht...
1    @shehryar_taseer that‚äôs üíø true , \ncorona...
2    tldr: not sars, possibly new coronavirus. diff...
3    disease outbreak news from the who: middle eas...
4    china - media: wsj says sources tell them myst...
Name: tweet, dtype: object

In [6]:
# Remove urls
def remove_urls(text):
    url_pattern = r'http\S+|www\S+|https\S+'
    return re.sub(url_pattern, '', text, flags=re.MULTILINE)
X = X.apply(remove_urls)

# Remove mentions
def remove_mentions(text):
    mention_pattern = r'@\w+'
    return re.sub(mention_pattern, ' ', text)
X = X.apply(remove_mentions)

X.head()

0         coronavirus | human coronavirus types | cdc 
1      that‚äôs üíø true , \ncorona virus \nswine ...
2    tldr: not sars, possibly new coronavirus. diff...
3    disease outbreak news from the who: middle eas...
4    china - media: wsj says sources tell them myst...
Name: tweet, dtype: object

In [7]:
# Remove newlines
X = X.replace(r'\n', ' ', regex=True)

# Remove special characters
def remove_special_characters(text):
    special_char_pattern = r'[^a-zA-Z0-9\s]'
    return re.sub(special_char_pattern, ' ', text)
X = X.apply(remove_special_characters)

# # Remove extra numbers
# def remove_numbers(text):
#     return re.sub(r'\d+', ' ', text)
# X = X.apply(remove_numbers)

# Remove extra spaces
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()
X = X.apply(remove_extra_spaces)

X.head()

0              coronavirus human coronavirus types cdc
1    that s true corona virus swine flue bird flu i...
2    tldr not sars possibly new coronavirus difficu...
3    disease outbreak news from the who middle east...
4    china media wsj says sources tell them mystery...
Name: tweet, dtype: object

In [8]:
# Dictionary adapted and expanded from https://www.kaggle.com/code/nmaguette/up-to-date-list-of-slangs-for-text-preprocessing
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "fr": "for real",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "finna": "finally",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gonna": "going to",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wanna" : "want to",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def replace_abbreviations(text):
    for abbr, full in abbreviations.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)
    return text
X = X.apply(replace_abbreviations)
X.head()

0              coronavirus human coronavirus types cdc
1    that s true corona virus swine flue bird flu i...
2    too long i did not read not sars possibly new ...
3    disease outbreak news from the who middle east...
4    china media wsj says sources tell them mystery...
Name: tweet, dtype: object

In [9]:
# Remove stop words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    word_tokens = word_tokenize(text)
    return ' '.join([word for word in word_tokens if word not in stop_words])

X = X.apply(remove_stop_words)

[nltk_data] Downloading package punkt_tab to /Users/ltree/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ltree/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    word_tokens = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in word_tokens])

X = X.apply(lemmatize_text)
X.head()

[nltk_data] Downloading package wordnet to /Users/ltree/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ltree/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0               coronavirus human coronavirus type cdc
1    true corona virus swine flue bird flu december...
2    long read sars possibly new coronavirus diffic...
3    disease outbreak news middle east respiratory ...
4    china medium wsj say source tell mystery pneum...
Name: tweet, dtype: object

In [11]:
new_df = pd.DataFrame({'tweet': X, 'label': y})
new_df.to_csv(f'data/clean/clean_COVIDSenti.csv', index=False)

In [13]:
# Apply to all separate files
indices = ['A', 'B', 'C']
def clean_text(df):
    X, y = df['tweet'], df['label']
    y = y.replace({'pos': 1, 'neu': 0, 'neg': -1})
    X = X.apply(lambda x: x.lower())
    X = X.apply(remove_urls)
    X = X.apply(remove_mentions)
    X = X.replace(r'\n', ' ', regex=True)
    X = X.apply(remove_special_characters)
    X = X.apply(remove_extra_spaces)
    X = X.apply(replace_abbreviations)
    X = X.apply(remove_stop_words)
    X = X.apply(lemmatize_text)
    return pd.DataFrame({'tweet': X, 'label': y})

for index in indices:
    df = pd.read_csv(f'data/COVIDSenti-{index}.csv')
    cleaned_df = clean_text(df)
    cleaned_df.to_csv(f'data/clean/clean_COVIDSenti-{index}.csv', index=False)